In [1]:
# MNIST 손글씨 숫자 데이터를 인식하는 심층 CNN
'''
3X3의 작은 필터를 사용한 합성곱 계층
활성화 함수는 ReLU
완전연결 계층 뒤에 드롭아웃 계층 사용
dam을 사용해 최적화
가중치 초깃값은 'He 초깃값'
'''

In [ ]:
# coding: utf-8
import sys, os
sys.path.append(os.pardir)  # 부모 디렉터리의 파일을 가져올 수 있도록 설정
import pickle
import numpy as np
from collections import OrderedDict
from common.layers import *


class DeepConvNet:
    """정확도 99% 이상의 고정밀 합성곱 신경망

    네트워크 구성은 아래와 같음
        conv - relu - conv- relu - pool -
        conv - relu - conv- relu - pool -
        conv - relu - conv- relu - pool -
        affine - relu - dropout - affine - dropout - softmax
    """
    def __init__(self, input_dim=(1, 28, 28),
                 conv_param_1 = {'filter_num':16, 'filter_size':3, 'pad':1, 'stride':1},
                 conv_param_2 = {'filter_num':16, 'filter_size':3, 'pad':1, 'stride':1},
                 conv_param_3 = {'filter_num':32, 'filter_size':3, 'pad':1, 'stride':1},
                 conv_param_4 = {'filter_num':32, 'filter_size':3, 'pad':2, 'stride':1},
                 conv_param_5 = {'filter_num':64, 'filter_size':3, 'pad':1, 'stride':1},
                 conv_param_6 = {'filter_num':64, 'filter_size':3, 'pad':1, 'stride':1},
                 hidden_size=50, output_size=10):
        # 가중치 초기화===========
        # 각 층의 뉴런 하나당 앞 층의 몇 개 뉴런과 연결되는가（TODO: 자동 계산되게 바꿀 것）
        pre_node_nums = np.array([1*3*3, 16*3*3, 16*3*3, 32*3*3, 32*3*3, 64*3*3, 64*4*4, hidden_size])
        wight_init_scales = np.sqrt(2.0 / pre_node_nums)  # ReLU를 사용할 때의 권장 초깃값(He 초깃값)
        
        self.params = {}
        pre_channel_num = input_dim[0]
        for idx, conv_param in enumerate([conv_param_1, conv_param_2, conv_param_3, conv_param_4, conv_param_5, conv_param_6]):
            self.params['W' + str(idx+1)] = wight_init_scales[idx] * np.random.randn(conv_param['filter_num'], pre_channel_num, conv_param['filter_size'], conv_param['filter_size'])
            self.params['b' + str(idx+1)] = np.zeros(conv_param['filter_num'])
            pre_channel_num = conv_param['filter_num']
        self.params['W7'] = wight_init_scales[6] * np.random.randn(64*4*4, hidden_size)
        self.params['b7'] = np.zeros(hidden_size)
        self.params['W8'] = wight_init_scales[7] * np.random.randn(hidden_size, output_size)
        self.params['b8'] = np.zeros(output_size)

        # 계층 생성===========
        self.layers = []
        self.layers.append(Convolution(self.params['W1'], self.params['b1'], 
                           conv_param_1['stride'], conv_param_1['pad']))
        self.layers.append(Relu())
        self.layers.append(Convolution(self.params['W2'], self.params['b2'], 
                           conv_param_2['stride'], conv_param_2['pad']))
        self.layers.append(Relu())
        self.layers.append(Pooling(pool_h=2, pool_w=2, stride=2))
        self.layers.append(Convolution(self.params['W3'], self.params['b3'], 
                           conv_param_3['stride'], conv_param_3['pad']))
        self.layers.append(Relu())
        self.layers.append(Convolution(self.params['W4'], self.params['b4'],
                           conv_param_4['stride'], conv_param_4['pad']))
        self.layers.append(Relu())
        self.layers.append(Pooling(pool_h=2, pool_w=2, stride=2))
        self.layers.append(Convolution(self.params['W5'], self.params['b5'],
                           conv_param_5['stride'], conv_param_5['pad']))
        self.layers.append(Relu())
        self.layers.append(Convolution(self.params['W6'], self.params['b6'],
                           conv_param_6['stride'], conv_param_6['pad']))
        self.layers.append(Relu())
        self.layers.append(Pooling(pool_h=2, pool_w=2, stride=2))
        self.layers.append(Affine(self.params['W7'], self.params['b7']))
        self.layers.append(Relu())
        self.layers.append(Dropout(0.5))
        self.layers.append(Affine(self.params['W8'], self.params['b8']))
        self.layers.append(Dropout(0.5))
        
        self.last_layer = SoftmaxWithLoss()

    def predict(self, x, train_flg=False):
        for layer in self.layers:
            if isinstance(layer, Dropout):
                x = layer.forward(x, train_flg)
            else:
                x = layer.forward(x)
        return x

    def loss(self, x, t):
        y = self.predict(x, train_flg=True)
        return self.last_layer.forward(y, t)

    def accuracy(self, x, t, batch_size=100):
        if t.ndim != 1 : t = np.argmax(t, axis=1)

        acc = 0.0

        for i in range(int(x.shape[0] / batch_size)):
            tx = x[i*batch_size:(i+1)*batch_size]
            tt = t[i*batch_size:(i+1)*batch_size]
            y = self.predict(tx, train_flg=False)
            y = np.argmax(y, axis=1)
            acc += np.sum(y == tt)

        return acc / x.shape[0]

    def gradient(self, x, t):
        # forward
        self.loss(x, t)

        # backward
        dout = 1
        dout = self.last_layer.backward(dout)

        tmp_layers = self.layers.copy()
        tmp_layers.reverse()
        for layer in tmp_layers:
            dout = layer.backward(dout)

        # 결과 저장
        grads = {}
        for i, layer_idx in enumerate((0, 2, 5, 7, 10, 12, 15, 18)):
            grads['W' + str(i+1)] = self.layers[layer_idx].dW
            grads['b' + str(i+1)] = self.layers[layer_idx].db

        return grads

    def save_params(self, file_name="params.pkl"):
        params = {}
        for key, val in self.params.items():
            params[key] = val
        with open(file_name, 'wb') as f:
            pickle.dump(params, f)

    def load_params(self, file_name="params.pkl"):
        with open(file_name, 'rb') as f:
            params = pickle.load(f)
        for key, val in params.items():
            self.params[key] = val

        for i, layer_idx in enumerate((0, 2, 5, 7, 10, 12, 15, 18)):
            self.layers[layer_idx].W = self.params['W' + str(i+1)]
            self.layers[layer_idx].b = self.params['b' + str(i+1)]

In [2]:
# 훈련용 코드 - 실행시키려면 반나절은 걸린다.

# coding: utf-8
import sys, os
sys.path.append(os.pardir)  # 부모 디렉터리의 파일을 가져올 수 있도록 설정
import numpy as np
import matplotlib.pyplot as plt
from dataset.mnist import load_mnist
from deep_convnet import DeepConvNet
from common.trainer import Trainer

(x_train, t_train), (x_test, t_test) = load_mnist(flatten=False)

network = DeepConvNet()  
trainer = Trainer(network, x_train, t_train, x_test, t_test,
                  epochs=20, mini_batch_size=100,
                  optimizer='Adam', optimizer_param={'lr':0.001},
                  evaluate_sample_num_per_epoch=1000)
trainer.train()

# 매개변수 보관
network.save_params("deep_convnet_params.pkl")
print("Saved Network Parameters!")

In [3]:
# MNIST 데이터셋에 대해서는 층을 아주 깊게 하지 않고도 (현시점에서는) 최고 수준의 결과가 나온다.
# 손글씨 숫자라는 문제가 비교적 단순해서 신경망 표현력을 극한까지 높일 필요가 없기 때문이다.
# 그러나 대규모 일반 사물 인식등의 문제는 훨씬 복잡하므로 층을 깊게 하면 정확도를 크게 올릴 수있다.

In [4]:
# 데이터 확장data augmentation

# 입력 이미지(훈련 이미지)를 알고리즘을 동원해 '인위적'으로 확장한다.
# 예를 들어 조금씩 회전, 이미지 세로로 이동 등, 미세한 변화를 주어 이미지 개수를 늘리는 것이다. 이러한 방식은 데이터가 몇 개 없을 때 특히 효과적인 수단이다.
# 이미지 일부를 잘라내는 crop, 좌우를 뒤집는 flip 등(단, 이미지 대칭성을 고려하지 않아도 될 때), 밝기 등이 외형 변화, 또는 확대 축소 등의 스케일 변화까지.

In [ ]:
# '층을 깊게 하는 것'이 왜 중요한가?

# 신경망의 매개 변수 수가 줄어든다.
'''
합성곱 연산에서의 필터 크기에 주목해 생각해보자.
5x5 필터로 구성된 합성곱 계층을 1회 반복하였을 때와, 3x3의 필터로 구성된 합성곱 계층을 2회 반복하였을 때를 생각해보자.
5x5 합성곱 계층은 출력 데이터의 노드 1개가 입력 데이터의 5x5 영역에서 도출된다.
3x3 합성곱 계층은 출력 데이터의 노드 1개가 입력 데이터의 3x3 영역에서 도출된다. 이를 2회 반복하면, 첫 입력 데이터가 5x5 영역과 같아진다(그림8-6 참조)
따라서 5x5 합성곱 연산 1회는 3x3 합성곱 연산 2회 수행으로 대체될 수 있다.
게다가, 전자는 매개변수 수가 25개(5x5)인 반면, 후자는 18개(2x3x3)이며, 매개변수 수는 층을 반복할수록 커지고 층이 깊어질수록 그 차이가 커진다.

작은 필터를 겹쳐 신경망을 깊게 할 때의 장점은, 이처럼 매개변수 수를 줄여 넓은 수용 영역receptive field을 소화할 수 있다는 데에 있다.
*수용영역 : 뉴런에 변화를 일으키는 국소적인 공간 영역
게다가 층을 거듭하면서 ReLU 등의 활성화 함수를 합성곱 계층에 끼워 신경망의 표현력이 개선된다. 활성화 함수가 신경망에 '비선형'적인 힘을 가하고, 이게 겹치면서 더 복잡한 표현도 할 수 있게 되므로.

또한, 신경망을 깊게 하면 학습해야할 문제를 계층적으로 분해할 수 있다.
개를 인식하는 사물인식 문제에서, 첫 층은 엣지가 있는지 없는지를 판단하고, 두 번째 층은 엣지가 어떤 모양인지 판단하고... 이런 식으로.
'''